<a href="https://colab.research.google.com/github/raisa1511/Raisabanu-555/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# -------------------------
# 1) Install dependencies
# -------------------------
!pip install --quiet tensorflow==2.16.1 scikit-learn matplotlib pandas numpy shap tqdm

# -------------------------
# 2) Imports and seeds
# -------------------------
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import shap
import json
import random

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

OUTDIR = "/content/forecast_outputs"
os.makedirs(OUTDIR, exist_ok=True)

# -------------------------
# 3) Synthetic dataset generator
# -------------------------
def generate_multivariate_series(n_points=5000, n_features=3, noise_std=0.2, seasonal_period=50):
    """
    Generate a synthetic multivariate time series with correlated features + target.
    - n_points: time steps
    - n_features: number of exogenous features (>=3)
    Returns a DataFrame with columns: feat_0 .. feat_{n_features-1}, target
    """
    t = np.arange(n_points)
    # base trend and common seasonality
    trend = 0.0005 * t  # gentle trend
    shared_season = 0.8 * np.sin(2 * np.pi * t / seasonal_period)
    data = {}
    for i in range(n_features):
        # each feature has own amplitude, phase, and noise plus shared signal
        amp = 0.5 + 0.5 * (i / (n_features - 1 + 1e-9))
        phase = i * 0.5
        individual_season = amp * np.sin(2 * np.pi * t / (seasonal_period * (1 + 0.1*i)) + phase)
        autoreg = 0.2 * np.roll(np.sin(0.02 * t + i), 1)  # weak auto-like pattern
        noise = np.random.normal(scale=noise_std, size=n_points)
        data[f"feat_{i}"] = trend + shared_season * (0.6 + 0.1*i) + individual_season + autoreg + noise

    # Build target as a correlated combination + its own seasonality + extra noise
    target = 0.4 * data["feat_0"] + 0.3 * data["feat_1"] + 0.2 * data["feat_2"]
    target += 0.5 * np.sin(2 * np.pi * t / (seasonal_period/2 + 1))  # different season for target
    target += np.random.normal(scale=noise_std * 1.2, size=n_points)
    df = pd.DataFrame(data)
    df["target"] = target
    return df

# Generate data
df = generate_multivariate_series(n_points=5000, n_features=4, noise_std=0.25, seasonal_period=60)
df.to_csv(os.path.join(OUTDIR, "synthetic_multivariate.csv"), index=False)
print("Generated data shape:", df.shape)
print(df.head())

# Quick plot (saved)
plt.figure(figsize=(12,5))
for c in df.columns[:4]:
    plt.plot(df[c], alpha=0.6, label=c)
plt.plot(df["target"], color='k', linewidth=1, label="target")
plt.legend()
plt.title("Synthetic multivariate series (first 4 features + target)")
plt.savefig(os.path.join(OUTDIR, "series_preview.png"))
plt.close()

# -------------------------
# 4) Preprocessing and windowing
# -------------------------
def make_windows(df, input_len=60, horizon=10, target_col="target"):
    """
    Create numpy arrays (X, y) for supervised learning.
    X shape: (samples, input_len, n_features)
    y shape: (samples, horizon)  # target forecasts only (can expand to multivariate targets)
    """
    features = [c for c in df.columns if c != target_col]
    Xs, ys = [], []
    arr_feats = df[features].values
    arr_target = df[target_col].values
    n = len(df)
    for start in range(0, n - input_len - horizon + 1):
        X = arr_feats[start:start+input_len]
        y = arr_target[start+input_len:start+input_len+horizon]
        Xs.append(X)
        ys.append(y)
    Xs = np.array(Xs)
    ys = np.array(ys)
    return Xs, ys, features

INPUT_LEN = 60
HORIZON = 10
X, y, feature_names = make_windows(df, input_len=INPUT_LEN, horizon=HORIZON)
print("Windowed shapes:", X.shape, y.shape)

# Split: train / val / test
n_samples = X.shape[0]
train_end = int(n_samples * 0.7)
val_end = int(n_samples * 0.85)
X_train, y_train = X[:train_end], y[:train_end]
X_val, y_val = X[train_end:val_end], y[train_end:val_end]
X_test, y_test = X[val_end:], y[val_end:]

# Scale features (fit on train)
feat_scaler = StandardScaler()
# reshape to 2D to fit
ft_train_2d = X_train.reshape(-1, X_train.shape[-1])
feat_scaler.fit(ft_train_2d)
# apply
def scale_X(X, scaler):
    s = X.reshape(-1, X.shape[-1])
    s = scaler.transform(s)
    return s.reshape(X.shape)
X_train_s = scale_X(X_train, feat_scaler)
X_val_s = scale_X(X_val, feat_scaler)
X_test_s = scale_X(X_test, feat_scaler)

# scale target
t_scaler = StandardScaler()
t_scaler.fit(y_train)  # shape: (samples, horizon) works fine
y_train_s = t_scaler.transform(y_train)
y_val_s = t_scaler.transform(y_val)
y_test_s = t_scaler.transform(y_test)

# Save scalers
import joblib
joblib.dump(feat_scaler, os.path.join(OUTDIR, "feat_scaler.joblib"))
joblib.dump(t_scaler, os.path.join(OUTDIR, "target_scaler.joblib"))

# -------------------------
# 5) Build model function
# -------------------------
def build_lstm_model(input_len, n_features, horizon, n_units=64, n_layers=2, dropout=0.2, lr=1e-3):
    """
    Build a stacked LSTM many-to-many model that outputs 'horizon' predictions.
    """
    inp = layers.Input(shape=(input_len, n_features), name="input_sequence")
    x = inp
    for i in range(n_layers - 1):
        x = layers.LSTM(n_units, return_sequences=True, name=f"lstm_{i+1}")(x)
        x = layers.Dropout(dropout)(x)
    # Final LSTM returns last output
    x = layers.LSTM(n_units, return_sequences=False, name=f"lstm_{n_layers}")(x)
    x = layers.Dropout(dropout)(x)
    out = layers.Dense(horizon, name="forecast_dense")(x)
    model = models.Model(inp, out)
    model.compile(optimizer=optimizers.Adam(learning_rate=lr), loss="mse", metrics=["mae"])
    return model

# Quick model summary
model = build_lstm_model(INPUT_LEN, X.shape[-1], HORIZON)
model.summary()

# -------------------------
# 6) Hyperparameter search (simple)
# -------------------------
def run_hp_search(hp_space, max_models=6, epochs=20):
    """
    Try combinations from hp_space (list of dicts). Train each, record val loss.
    Return a list of dicts with results and top-k models saved.
    """
    results = []
    for i in range(len(hp_space)):
        if i >= max_models:
            break
        hp = hp_space[i]
        print(f"\n=== HP trial {i+1}/{min(max_models, len(hp_space))}: {hp}")
        m = build_lstm_model(INPUT_LEN, X.shape[-1], HORIZON,
                             n_units=hp["n_units"], n_layers=hp["n_layers"],
                             dropout=hp["dropout"], lr=hp["lr"])
        cb = [callbacks.EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)]
        hist = m.fit(X_train_s, y_train_s, validation_data=(X_val_s, y_val_s),
                     epochs=epochs, batch_size=hp["batch_size"], callbacks=cb, verbose=0)
        best_val_loss = min(hist.history["val_loss"])
        print(f"Best val_loss: {best_val_loss:.6f}")
        # Save model
        model_path = os.path.join(OUTDIR, f"model_hp_{i+1}.keras") # Changed to .keras
        m.save(model_path)
        results.append({"hp": hp, "val_loss": float(best_val_loss), "model_path": model_path})
    # sort by val_loss
    results = sorted(results, key=lambda x: x["val_loss"])
    # save results to json
    with open(os.path.join(OUTDIR, "hp_results.json"), "w") as f:
        json.dump(results, f, indent=2)
    return results

# Define a compact hp space to try (you can expand)
hp_space = [
    {"n_units": 32, "n_layers": 1, "dropout": 0.1, "lr": 1e-3, "batch_size": 64},
    {"n_units": 64, "n_layers": 2, "dropout": 0.2, "lr": 5e-4, "batch_size": 64},
    {"n_units": 128, "n_layers": 2, "dropout": 0.3, "lr": 1e-3, "batch_size": 32},
    {"n_units": 64, "n_layers": 3, "dropout": 0.2, "lr": 1e-4, "batch_size": 32},
    {"n_units": 32, "n_layers": 2, "dropout": 0.15, "lr": 3e-4, "batch_size": 128},
    {"n_units": 128, "n_layers": 1, "dropout": 0.25, "lr": 5e-4, "batch_size": 64}
]
results = run_hp_search(hp_space, max_models=6, epochs=25)
print("\nTop 5 HP results (val_loss asc):")
for r in results[:5]:
    print(r)

# Save top-5 hyperparameters in human-readable form
with open(os.path.join(OUTDIR, "top5_hyperparameters.txt"), "w") as f:
    for i, r in enumerate(results[:5]):
        f.write(f"Rank {i+1}: val_loss={r['val_loss']}\nHP: {r['hp']}\nModel: {r['model_path']}\n\n")

# -------------------------
# 7) Evaluate best model on test set
# -------------------------
best = results[0]
# Explicitly pass custom objects for standard metrics/losses if there's a deserialization issue
custom_objects = {"mse": tf.keras.losses.MeanSquaredError(), "mae": tf.keras.metrics.MeanAbsoluteError()}
best_model = tf.keras.models.load_model(best["model_path"], custom_objects=custom_objects)
# Predictions (scaled)
y_pred_test_s = best_model.predict(X_test_s)
# inverse scale
y_pred_test = t_scaler.inverse_transform(y_pred_test_s)
y_test_orig = y_test  # already original

# Compute RMSE and MAE per horizon and averaged
rmse_per_h = np.sqrt(np.mean((y_pred_test - y_test_orig)**2, axis=0))
mae_per_h = np.mean(np.abs(y_pred_test - y_test_orig), axis=0)
rmse_avg = np.mean(rmse_per_h)
mae_avg = np.mean(mae_per_h)
metrics = {"rmse_per_horizon": rmse_per_h.tolist(), "mae_per_horizon": mae_per_h.tolist(),
           "rmse_avg": float(rmse_avg), "mae_avg": float(mae_avg)}
print("Test metrics:", metrics)
with open(os.path.join(OUTDIR, "test_metrics.json"), "w") as f:
    json.dump(metrics, f, indent=2)

# Plot predictions for a few test windows and save
n_plot = 5
for i in range(n_plot):
    plt.figure(figsize=(8,3))
    plt.plot(y_test_orig[i], label="true")
    plt.plot(y_pred_test[i], label="pred")
    plt.title(f"Test sample {i} (horizon={HORIZON})")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(OUTDIR, f"test_pred_{i}.png"))
    plt.close()

# -------------------------
# 8) Explainability with SHAP
# -------------------------
def run_shap_explain(model, X_background, X_to_explain, feature_names, horizon_index=0):
    """
    Compute SHAP values explaining the model's output at a particular horizon index.
    - model: Keras model that maps (batch, input_len, n_features) -> horizon outputs
    - X_background: small background dataset (shape: (k, input_len, n_features))
    - X_to_explain: points to explain
    Returns shap_values array for features across input timesteps (same shape as X_to_explain).
    """
    try:
        explainer = shap.GradientExplainer((model.input, model.output[:, horizon_index]), X_background)
        shap_values = explainer.shap_values(X_to_explain)
        # shap_values shape for GradientExplainer: list-like; convert appropriately
        return shap_values
    except Exception as e:
        print("GradientExplainer failed, falling back to KernelExplainer:", e)
        # Use KernelExplainer on a reduced representation: flatten input
        model_flat = lambda x: model.predict(x.reshape((-1, INPUT_LEN, X.shape[-1])))[:, horizon_index]
        # Flatten background and to-explain
        bg = X_background.reshape(X_background.shape[0], -1)
        tx = X_to_explain.reshape(X_to_explain.shape[0], -1)
        expl = shap.KernelExplainer(model_flat, bg)
        shap_vals = expl.shap_values(tx, nsamples=100)
        # reshape back to (samples, input_len, n_features)
        shap_vals = np.array(shap_vals).reshape(tx.shape[0], INPUT_LEN, X.shape[-1])
        return shap_vals

# Choose a small background set from train
bg_idx = np.random.choice(len(X_train_s), size=100, replace=False)
X_bg = X_train_s[bg_idx]
# Points to explain: first 10 test samples
X_explain = X_test_s[:10]
# For a particular forecast horizon (e.g., horizon_index=0 => immediately next)
h_idx = 0
shap_vals = run_shap_explain(best_model, X_bg, X_explain, feature_names, horizon_index=h_idx)

# shap_vals may be in different formats; normalize into array shape (samples, input_len, n_features)
if isinstance(shap_vals, list):
    # shap.GradientExplainer often returns an array-like inside a list
    shap_arr = np.array(shap_vals[0])
else:
    shap_arr = np.array(shap_vals)
print("SHAP array shape:", shap_arr.shape)

# Aggregate importance across timesteps for each feature (mean absolute)
feature_imp = np.mean(np.abs(shap_arr), axis=(0,1)) if shap_arr.ndim==3 else np.mean(np.abs(shap_arr))
# if array is (samples, input_len, n_feat) reduce sample & time dims
if shap_arr.ndim == 3:
    feat_imp_per_feature = np.mean(np.abs(shap_arr), axis=(0,1))  # shape (n_features,)
else:
    feat_imp_per_feature = feature_imp

# Create bar plot
plt.figure(figsize=(8,4))
plt.bar(feature_names, feat_imp_per_feature)
plt.title(f"Mean |SHAP| per feature (horizon_index={h_idx})")
plt.tight_layout()
plt.savefig(os.path.join(OUTDIR, f"shap_feature_importance_h{h_idx}.png"))
plt.close()

# Save shap raw values for later analysis
np.save(os.path.join(OUTDIR, f"shap_values_h{h_idx}.npy"), shap_arr)

# -------------------------
# 9) Summary report (text)
# -------------------------
report = {
    "project": "Advanced Time Series Forecasting — synthetic dataset + LSTM",
    "dataset": {
        "n_points": int(len(df)),
        "n_features": int(X.shape[-1]),
        "input_len": INPUT_LEN,
        "horizon": HORIZON,
        "file": "synthetic_multivariate.csv"
    },
    "best_hyperparams": results[0]["hp"],
    "test_metrics": metrics,
    "saved_outputs_dir": OUTDIR,
    "plots": [
        "series_preview.png",
        *[f"test_pred_{i}.png" for i in range(n_plot)],
        f"shap_feature_importance_h{h_idx}.png"
    ]
}
with open(os.path.join(OUTDIR, "summary_report.json"), "w") as f:
    json.dump(report, f, indent=2)

Generated data shape: (5000, 5)
     feat_0    feat_1    feat_2    feat_3    target
0  0.019477  0.300496  0.730121  1.089341  0.348450
1  0.068372  0.487759  0.910682  1.095773  0.541459
2  0.370675  0.263505  0.933635  1.186954  0.330271
3  0.693092  0.736007  1.199127  1.466429  1.197374
4  0.354056  1.101616  1.551080  1.092907  0.697695
Windowed shapes: (4931, 60, 4) (4931, 10)


Model: "functional_47"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_sequence (InputLayer)     │ (None, 60, 4)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 60, 64)         │        17,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_84 (Dropout)            │ (None, 60, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_85 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ forecast_dense (Dense)          │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,338 (200.54 KB)

 Trainable params: 51,338 (200.54 KB)

 Non-trainable params: 0 (0.00 B)


=== HP trial 1/6: {'n_units': 32, 'n_layers': 1, 'dropout': 0.1, 'lr': 0.001, 'batch_size': 64}
Best val_loss: 0.503963

=== HP trial 2/6: {'n_units': 64, 'n_layers': 2, 'dropout': 0.2, 'lr': 0.0005, 'batch_size': 64}
Best val_loss: 0.460516

=== HP trial 3/6: {'n_units': 128, 'n_layers': 2, 'dropout': 0.3, 'lr': 0.001, 'batch_size': 32}
Best val_loss: 0.424864

=== HP trial 4/6: {'n_units': 64, 'n_layers': 3, 'dropout': 0.2, 'lr': 0.0001, 'batch_size': 32}
Best val_loss: 0.433126

=== HP trial 5/6: {'n_units': 32, 'n_layers': 2, 'dropout': 0.15, 'lr': 0.0003, 'batch_size': 128}
Best val_loss: 0.500349

=== HP trial 6/6: {'n_units': 128, 'n_layers': 1, 'dropout': 0.25, 'lr': 0.0005, 'batch_size': 64}
Best val_loss: 0.439606

Top 5 HP results (val_loss asc):
{'hp': {'n_units': 128, 'n_layers': 2, 'dropout': 0.3, 'lr': 0.001, 'batch_size': 32}, 'val_loss': 0.42486366629600525, 'model_path': '/content/forecast_outputs/model_hp_3.keras'}
{'hp': {'n_units': 64, 'n_layers': 3, 'dropout': 0.

In [12]:
import json
import os

# Assuming OUTDIR is defined from the previous cells
results_path = os.path.join(OUTDIR, "hp_results.json")

if os.path.exists(results_path):
    with open(results_path, "r") as f:
        all_results = json.load(f)
    print("Full Hyperparameter Search Results (sorted by val_loss):")
    for i, res in enumerate(all_results):
        print(f"Rank {i+1}: Val Loss = {res['val_loss']:.6f}, HP = {res['hp']}")
else:
    print(f"Error: hp_results.json not found at {results_path}")


Full Hyperparameter Search Results (sorted by val_loss):
Rank 1: Val Loss = 0.424864, HP = {'n_units': 128, 'n_layers': 2, 'dropout': 0.3, 'lr': 0.001, 'batch_size': 32}
Rank 2: Val Loss = 0.433126, HP = {'n_units': 64, 'n_layers': 3, 'dropout': 0.2, 'lr': 0.0001, 'batch_size': 32}
Rank 3: Val Loss = 0.439606, HP = {'n_units': 128, 'n_layers': 1, 'dropout': 0.25, 'lr': 0.0005, 'batch_size': 64}
Rank 4: Val Loss = 0.460516, HP = {'n_units': 64, 'n_layers': 2, 'dropout': 0.2, 'lr': 0.0005, 'batch_size': 64}
Rank 5: Val Loss = 0.500349, HP = {'n_units': 32, 'n_layers': 2, 'dropout': 0.15, 'lr': 0.0003, 'batch_size': 128}
Rank 6: Val Loss = 0.503963, HP = {'n_units': 32, 'n_layers': 1, 'dropout': 0.1, 'lr': 0.001, 'batch_size': 64}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')